In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import mysql.connector as sql
import csv
from PIL import Image, ImageTk

Log in Window

In [2]:
class LoginWindow:
    def __init__(self, root, on_successful_login):
        self.root = root
        self.root.title("Login to Swiggy Restaurant Data")
        self.on_successful_login = on_successful_login

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg_LOGIN.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self.root, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        # Create the login frame
        self.login_frame = tk.Frame(self.root, bg='white')
        self.login_frame.pack(padx=10, pady=175)

        # Username field
        tk.Label(self.login_frame, text="Username:", bg='white').grid(row=0, column=0, padx=5, pady=5)
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.grid(row=0, column=1, padx=5, pady=5)

        # Password field
        tk.Label(self.login_frame, text="Password:", bg='white').grid(row=1, column=0, padx=5, pady=5)
        self.password_entry = tk.Entry(self.login_frame, show="*")
        self.password_entry.grid(row=1, column=1, padx=5, pady=5)

        # Login button
        tk.Button(self.login_frame, text="Login", command=self.attempt_login).grid(row=2, column=0, columnspan=2, pady=10)

    def attempt_login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Read credentials from CSV file
        credentials = {}
        try:
            with open("credentials.csv", "r") as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    credentials[row["user"]] = row["password"]
        except FileNotFoundError:
            messagebox.showerror("Error", "credentials.csv file not found.")
            self.root.quit()
            return
        except Exception as e:
            messagebox.showerror("Error", f"Failed to read credentials: {e}")
            self.root.quit()
            return

        # Check if the username and password match
        if username in credentials and credentials[username] == password:
            self.login_frame.destroy()
            self.on_successful_login(username, password)
        else:
            messagebox.showerror("Login Failed", "Invalid username or password.")

All Queries

In [3]:
class SwiggyApp:
    def __init__(self, root, username, password, show_login):
        self.root = root
        self.root.title("Swiggy Restaurant Data")
        self.show_login = show_login

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self.root, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        # Attempt to connect to the MySQL database using the provided credentials
        try:
            self.mydb = sql.connect(
                host="192.168.194.204",
                user="sagnik",
                passwd="remote",
                database="swiggy_python",
                port="3306"
            )
            self.mycursor = self.mydb.cursor()
        except sql.Error as e:
            messagebox.showerror("Database Error", f"Failed to connect to database: {e}")
            self.root.quit()
            return

        # Create the main frame and label
        self.main_frame = tk.Frame(self.root, bg='white')
        self.main_frame.pack(padx=10, pady=150)

        tk.Label(self.main_frame, text="Select a query:", bg='white').pack()

        # Add buttons for each query
        tk.Button(self.main_frame, text="City-wise Restaurant Count", command=self.query1).pack(fill='x')
        tk.Button(self.main_frame, text="Top 10 Restaurants by Avg Cost for 2", command=self.query2).pack(fill='x')
        tk.Button(self.main_frame, text="Top 10 Restaurants by Avg Votes", command=self.query3).pack(fill='x')
        tk.Button(self.main_frame, text="Top 10 Restaurants by Rating", command=self.query4).pack(fill='x')
        tk.Button(self.main_frame, text="Rating by Delivery Availability", command=self.query5).pack(fill='x')
        tk.Button(self.main_frame, text="Avg Cost for 2 by Cuisine", command=self.query6).pack(fill='x')

        # Add logout button
        tk.Button(self.main_frame, text="Logout", command=self.logout).pack(fill='x', pady=5)

    def logout(self):
        """Handle logout by closing the database connection and returning to login page"""
        self.close_connection()
        self.main_frame.destroy()
        self.show_login()

    def show_main_window(self):
        self.root.deiconify()

    def query1(self):
        self.root.withdraw()
        title = "City-wise Restaurant Count"
        sql_query = "SELECT City, COUNT(*) AS Count FROM swiggy_source GROUP BY City ORDER BY Count DESC"
        columns = ["City", "Count"]
        widths = [15, 10]
        self.execute_query(title, sql_query, columns, widths)

    def query2(self):
        self.root.withdraw()
        title = "Top 10 Restaurants by Avg Cost for 2"
        sql_query = "SELECT city, restaurant_name, avg_cost_for_two FROM swiggy_source ORDER BY avg_cost_for_two DESC LIMIT 10"
        columns = ["City", "Restaurant Name", "Avg Cost for 2"]
        widths = [15, 50, 15]
        self.execute_query(title, sql_query, columns, widths)

    def query3(self):
        self.root.withdraw()
        title = "Top 10 Restaurants by Avg Votes"
        sql_query = "SELECT City, Restaurant_Name, AVG(Votes) AS Votes FROM swiggy_source GROUP BY Restaurant_Name, City ORDER BY Votes DESC LIMIT 10"
        columns = ["City", "Restaurant Name", "Avg Votes"]
        widths = [15, 50, 15]
        self.execute_query(title, sql_query, columns, widths)

    def query4(self):
        self.root.withdraw()
        title = "Top 10 Restaurants by Rating"
        sql_query = "SELECT city City, restaurant_name `Restaurant Name`, rating_stars_out_of_5 Rating FROM swiggy_source ORDER BY city, Rating DESC LIMIT 10"
        columns = ["City", "Restaurant Name", "Rating"]
        widths = [15, 50, 10]
        self.execute_query(title, sql_query, columns, widths)

    def query5(self):
        self.root.withdraw()
        title = "Rating by Delivery Availability"
        sql_query = "SELECT City, has_online_delivery `Online Delivery`, rating_stars_out_of_5 Rating FROM swiggy_source"
        columns = ["City", "Online Delivery", "Rating"]
        widths = [15, 20, 10]
        self.execute_query(title, sql_query, columns, widths)

    def query6(self):
        self.root.withdraw()
        title = "Avg Cost for 2 by Cuisine"
        sql_query = "SELECT City, Cuisines Cuisine, avg_cost_for_two `Average Cost for two` FROM swiggy_source"
        columns = ["City", "Cuisine", "Average Cost for two"]
        widths = [15, 20, 20]
        self.execute_query(title, sql_query, columns, widths)

    def execute_query(self, title, sql_query, columns, widths):
        try:
            self.mycursor.execute(sql_query)
            result = self.mycursor.fetchall()
            ResultsWindow(self, title, columns, result, widths)
        except sql.Error as e:
            messagebox.showerror("Query Error", f"Failed to execute query: {e}")
            self.show_main_window()

    def close_connection(self):
        if hasattr(self, 'mydb') and self.mydb.is_connected():
            self.mydb.close()
            self.mycursor.close()

Output Window

In [4]:
class ResultsWindow(tk.Toplevel):
    def __init__(self, main_app, title, columns, data, widths):
        tk.Toplevel.__init__(self)
        self.main_app = main_app
        self.title(title)
        self.geometry("1280x720")

        # Load and set background image
        try:
            self.bg_image = ImageTk.PhotoImage(Image.open("bg.jpg").resize((1280, 720), Image.Resampling.LANCZOS))
            self.bg_label = tk.Label(self, image=self.bg_image)
            self.bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load background image: {e}")

        if not data:
            tk.Label(self, text="No results found.", bg='white').pack(pady=20)
        else:
            tree_frame = tk.Frame(self, bg='white')
            tree_frame.pack(expand=True, fill='both')

            tree = ttk.Treeview(tree_frame, columns=columns, show='headings')
            for col, width in zip(columns, widths):
                tree.heading(col, text=col)
                tree.column(col, width=width * 10)

            for row in data:
                tree.insert('', 'end', values=row)

            scrollbar = tk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=scrollbar.set)
            tree.pack(side='left', expand=True, fill='both')
            scrollbar.pack(side='right', fill='y')

        back_button = tk.Button(self, text="Back", command=self.go_back)
        back_button.pack(pady=10)

        self.protocol("WM_DELETE_WINDOW", self.go_back)

    def go_back(self):
        self.destroy()
        self.main_app.show_main_window()

Main Function

In [5]:
def main():
    root = tk.Tk()
    root.geometry("1280x720")  # Set default window size

    def show_login():
        for widget in root.winfo_children():
            widget.destroy()
        LoginWindow(root, lambda username, password: SwiggyApp(root, username, password, show_login))

    # Start with the login window
    show_login()

    root.protocol("WM_DELETE_WINDOW", lambda: [root.destroy()])
    root.mainloop()

In [6]:
if __name__ == "__main__":
    main()